# Conway's Game of Life - Unlimited Editition - 3 kyu

[Kata Source](http://www.codewars.com/kata/52423db9add6f6fc39000354)

Given a 2D array and a number of generations, compute n timesteps of Conway's Game of Life.

The rules of the game are: 1. Any live cell with fewer than two live neighbours dies, as if caused by underpopulation. 2. Any live cell with more than three live neighbours dies, as if by overcrowding. 3. Any live cell with two or three live neighbours lives on to the next generation. 4. Any dead cell with exactly three live neighbours becomes a live cell.

Each cell's neighborhood is the 8 cells immediately around it. The universe is infinite in both the x and y dimensions and all cells are initially dead - except for those specified in the arguments. The return value should be a 2d array cropped around all of the living cells. (If there are no living cells, then return [[]].)

For illustration purposes, 0 and 1 will be represented as ░░ and ▓▓ blocks respectively. You can take advantage of the htmlize function to get a text representation of the universe: eg:

In [ ]:
print htmlize(cells)

# Solution

We need a representation of the board that we can iterate in an arbitrary order and that can be very large and sparse. A matrix or table will not do as it will contain a substential amount of zeros. We will use the keys of a dictionary for this. 

We need a function to convert a table to a dictionary.

In [ ]:
def convert_table_to_dict(table):
    
    ni = len(table)
    nj = len(table[0])
    
    dict = {}
    for i in range(ni):
      for j in range(nj):
        if table[i][j] == 1:
          dict[(i,j)] = None
          
    return dict

We also need a function to convert a dictionary to a cropped table.

In [ ]:
def convert_dict_to_table(dict):

  keys = dict.keys()
  
  if not keys:
    mi = mj = ni = nj = 0
  else:
    mi = min(map(lambda x: x[0], keys))
    mj = min(map(lambda x: x[1], keys))
    ni = max(map(lambda x: x[0], keys))
    nj = max(map(lambda x: x[1], keys))
  
  table = [[0 for j in range(nj-mj+1)] for i in range(ni-mi+1)]
  
  for (i,j) in keys:
    table[i-mi][j-mj] = 1
  
  return table

For each generations... 
* We iterate over all the cells currently active and create a set containing those along with their neighbours. 
* We then iterate over all of those, count the neighbours that are alive, and decide whether the cell should be alive or not in the next generation.

In [ ]:
from itertools import product

def get_generation(cells, generations):

  cells_dict = convert_table_to_dict(cells)

  for n in range(generations):
    
    stack = {}
    cells_new = {}
    
    for (i,j) in cells_dict:
      for (di, dj) in product((-1,0,1), (-1,0,1)):
        stack[(i+di,j+dj)] = None
    
    for (i,j) in stack:
      neighbours = 0
      for (di, dj) in product((-1,0,1), (-1,0,1)):
        if (i+di,j+dj) in cells_dict and (di,dj) != (0,0):
          neighbours += 1
        
      if neighbours == 3:
        cells_new[(i,j)] = None
      elif neighbours == 2 and (i,j) in cells_dict:
        cells_new[(i,j)] = None
          
    cells_dict = cells_new
          
  return convert_dict_to_table(cells_dict)

# Test

Here's the provided tests.

In [ ]:
def htmlize(array):
    s = []
    for row in array:
        for cell in row:
            s.append('▓▓' if cell else '░░')
        s.append('<br/>')
    return ''.join(s)

start = [[1,0,0],
         [0,1,1],
         [1,1,0]]
end   = [[0,1,0],
         [0,0,1],
         [1,1,1]]
         
test.describe('Glider<br/>' + htmlize(start))
test.it('Glider 1')

resp = get_generation(start, 1)
test.expect(resp == end, 'Got<br/>' + htmlize(resp) + '<br/>instead of<br/>' + htmlize(end))